# Yahoo Finance 網站內國際金融與股市歷史資料下載

In [ ]:
# 可以去掉 python 輸出時，因為軟體版本所引起的警告的警告。
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import re         # 正規表達式(regular expression)模組
import sys        # 系統模組，內含很多函式方法和變數，用來處理Python執行時配置以及資源，從而可以與當前程式之外的系統環境互動
import time       # 時間模組
import datetime   # 日期模組
import requests   # 網路連線模組

In [ ]:
def split_crumb_store(v):

    return v.split(':')[2].strip('"')

In [ ]:
def find_crumb_store(lines):

    # Looking for
    # ,"CrumbStore":{"crumb":"9q.A4D1c.b9

    for l in lines:

        if re.findall(r'CrumbStore', l):

            return l

    print("Did not find CrumbStore")

In [ ]:
 # 取得並保存 cookie 
def get_cookie_value(r):
    
    return {'B': r.cookies['B']}

In [ ]:
def get_data(symbol, start_date, end_date, cookie, crumb):

    filename = '%s.csv' % (symbol)

    url = "https://query1.finance.yahoo.com/v7/finance/download/%s?period1=%s&period2=%s&interval=1d&events=history&crumb=%s" % (symbol, start_date, end_date, crumb)

    response = requests.get(url, cookies = cookie)

    with open (filename, 'wb') as handle:

        for block in response.iter_content(1024):

            handle.write(block)

In [ ]:
def get_page_data(symbol):

    url = "https://finance.yahoo.com/quote/%s/?p=%s" % (symbol, symbol)

    r = requests.get(url)

    #print(r.text)

    try:

        cookie = get_cookie_value(r)
        
        #print(cookie)

    except KeyError as e:

        print(">> Error Message : Symbol name [%s] not found!\n" % symbol)

        sys.exit()

    else:
        
        # Code to replace possible \u002F value
        # ,"CrumbStore":{"crumb":"FWP\u002F5EFll3U"
        # FWP\u002F5EFll3U
        lines = r.content.decode('unicode-escape').strip().replace('}', '\n')

        return cookie, lines.split('\n')

In [ ]:
def get_cookie_crumb(symbol):

    cookie, lines = get_page_data(symbol)

    crumb = split_crumb_store(find_crumb_store(lines))

    return cookie, crumb

In [ ]:
def get_now_epoch():

    # @see https://www.linuxquestions.org/questions/programming-9/python-datetime-to-epoch-4175520007/#post5244109

    return int(time.time())

In [ ]:
def download_quotes(symbol):

    start_date = 0

    end_date = get_now_epoch()

    cookie, crumb = get_cookie_crumb(symbol)

    get_data(symbol, start_date, end_date, cookie, crumb)

### 以下為主程式區

In [ ]:
if __name__ == '__main__':

    counter = 0
    
    while(True):

        print()

        counter += 1
        
        symbol = input('[#%d] Enter the symbol: ' % counter)

        print("--------------------------------------------------")

        print("Downloading %s to %s.csv" % (symbol, symbol))

        download_quotes(symbol)

        print("--------------------------------------------------")

        another_run = input('\nContinue? (y/n)')

        if (another_run == 'n' or another_run == 'N' or another_run == 'no' or another_run == 'No'):

            print('\n* Download %d stock quotes from Yahoo finance!\n' % counter) 

            break
        